In [1]:
%pwd

'c:\\Users\\HP\\Desktop\\ML\\Text Summerization\\Text_summarizer_NLP\\research'

In [3]:
import os
os.chdir("../")

In [28]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    weight_decay:float
    logging_steps : int
    evaluation_strategy:str
    eval_steps:int
    save_steps:float
    gradient_accumulation_steps:int

In [29]:
from Text_Summarizer.constant import *
from Text_Summarizer.utils.common import read_yaml,create_directories

In [30]:
class ConfigurationManager:
    def __init__(self,config_file = CONFIG_FILE_PATH,param_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file)
        self.param = read_yaml(param_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:

        config = self.config.model_trainer
        params = self.param.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
                root_dir = config.root_dir,
                data_path = config.data_path,
                model_ckpt =  config.model_ckpt,
                num_train_epochs = params.num_train_epochs,
                warmup_steps = params.warmup_steps,
                per_device_train_batch_size = params.per_device_train_batch_size,
                weight_decay = params.weight_decay,
                logging_steps = params.logging_steps,
                evaluation_strategy = params.evaluation_strategy,
                eval_steps = params.eval_steps,
                save_steps = params.save_steps,
                gradient_accumulation_steps = params.gradient_accumulation_steps
            
        )

        return model_trainer_config

In [31]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [32]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [36]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-31 23:49:22,620: INFO: common: yaml file: C:\Users\HP\Desktop\ML\Text Summerization\Text_summarizer_NLP\config\config.yaml loaded successfully]
[2024-08-31 23:49:23,117: INFO: common: yaml file: C:\Users\HP\Desktop\ML\Text Summerization\Text_summarizer_NLP\params.yaml loaded successfully]
[2024-08-31 23:49:23,207: INFO: common: created directory at: artifacts]
[2024-08-31 23:49:23,207: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`